In [ ]:
import numpy as np
import xarray as xr
import scipy.stats as st
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import requests
import os,errno
import sys
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER,LATITUDE_FORMATTER
import matplotlib.ticker as mticker
import cartopy.feature as cfeature
import datetime as dt
import pandas as pd
import time
import urllib.request
import metpy.calc as mpcalc
import salem
import scipy.optimize as opt
import warnings
import geopy.distance
import matplotlib as mpl
import matplotlib.colors as mcolors
from matplotlib.lines import Line2D
import matplotlib.backends.backend_pdf

from scipy.ndimage.measurements import label
from scipy.ndimage import binary_dilation

warnings.filterwarnings('ignore')

%matplotlib inline

dir_data='/Users/ahenny/'
dir1='/Volumes/LaCie/'
dir2='/Volumes/Extreme Pro/'
dir3='/Volumes/My Passport/day/'

In [ ]:
ds=xr.open_dataset('/Users/ahenny/Downloads/REGEN_AllStns_V1-2019_1950-2016_QualityMask.nc')
quality=ds['p']

dk=xr.Dataset()
dk['quality']=(('latitude','longitude'),quality.values)
dk.coords['latitude']=quality.lat.values
dk.coords['longitude']=quality.lon.values
dk['latitude'].attrs["units"]='degrees_north'
dk['longitude'].attrs["units"]='degrees_east'
quality=dk['quality']
print(quality)

ds=xr.open_dataset('/Users/ahenny/Downloads/HadEX3_61-90_R95p_ANN.nc')
precip=ds['R95p'][79:,:,:]

lon_east=[x for x in precip.longitude.values if x>=180.]
lon_west=[x for x in precip.longitude.values if x<180.]
l_east=precip.sel(longitude=lon_east)
l_west=precip.sel(longitude=lon_west)
l_east['longitude']=[x-360. for x in lon_east]
precip=xr.concat([l_east,l_west],dim='longitude')

lons_east=[x for x in precip.longitude.values if x>=90]
lons_west=[x for x in precip.longitude.values if x<-90]
precip_east=precip.sel(longitude=lons_east)
precip_west=precip.sel(longitude=lons_west)
precip_east['longitude']=[x-360. for x in lons_east]
precip_west['longitude']=[x+360. for x in lons_west]
precip=xr.concat([precip_east,precip,precip_west],dim='longitude')

quality=quality.fillna(0)
quality=quality.interp_like(precip)

In [ ]:
ds=xr.open_dataset('/Users/ahenny/Downloads/HadEX3_PRCPTOT_ANN.nc')
precip=ds['PRCPTOT'][49:,:,:]

#ds=xr.open_dataset('/Users/ahenny/Downloads/HadEX3_61-90_R95p_ANN.nc')
#precip=ds['R95p'][49:,:,:]

#ds=xr.open_dataset('/Users/ahenny/Downloads/HadEX3_61-90_R99p_ANN.nc')
#precip=ds['R99p'][49:,:,:]

lon_east=[x for x in precip.longitude.values if x>=180.]
lon_west=[x for x in precip.longitude.values if x<180.]
l_east=precip.sel(longitude=lon_east)
l_west=precip.sel(longitude=lon_west)
l_east['longitude']=[x-360. for x in lon_east]
precip=xr.concat([l_east,l_west],dim='longitude')

defined1=xr.ones_like(precip).sum(dim='time')
defined=xr.ones_like(precip).where(precip>=0).sum(dim='time')
ratio_all1=defined/defined1

In [ ]:
ds=xr.open_dataset('/Users/ahenny/Downloads/HadEX3_PRCPTOT_ANN.nc')
precip=ds['PRCPTOT'][39:,:,:]

#ds=xr.open_dataset('/Users/ahenny/Downloads/HadEX3_61-90_R95p_ANN.nc')
#precip=ds['R95p'][39:,:,:]

#ds=xr.open_dataset('/Users/ahenny/Downloads/HadEX3_61-90_R99p_ANN.nc')
#precip=ds['R99p'][39:,:,:]

lon_east=[x for x in precip.longitude.values if x>=180.]
lon_west=[x for x in precip.longitude.values if x<180.]
l_east=precip.sel(longitude=lon_east)
l_west=precip.sel(longitude=lon_west)
l_east['longitude']=[x-360. for x in lon_east]
precip=xr.concat([l_east,l_west],dim='longitude')



lons_east=[x for x in precip.longitude.values if x>=90]
lons_west=[x for x in precip.longitude.values if x<-90]
precip_east=precip.sel(longitude=lons_east)
precip_west=precip.sel(longitude=lons_west)
precip_east['longitude']=[x-360. for x in lons_east]
precip_west['longitude']=[x+360. for x in lons_west]
precip=xr.concat([precip_east,precip,precip_west],dim='longitude')
print(precip)
sys.exit()

defined1=xr.ones_like(precip).sum(dim='time')
defined=xr.ones_like(precip).where(precip>=0).sum(dim='time')
ratio_all2=defined/defined1

In [ ]:
#ds=xr.open_dataset('/Users/ahenny/Downloads/HadEX3_PRCPTOT_ANN.nc')
#precip=ds['PRCPTOT'][:,:,:]

#ds=xr.open_dataset('/Users/ahenny/Downloads/HadEX3_61-90_R95p_ANN.nc')
#precip=ds['R95p'][79:,:,:]

ds=xr.open_dataset('/Users/ahenny/Downloads/HadEX3_61-90_R99p_ANN.nc')
precip=ds['R99p'][79:,:,:]

lon_east=[x for x in precip.longitude.values if x>=180.]
lon_west=[x for x in precip.longitude.values if x<180.]
l_east=precip.sel(longitude=lon_east)
l_west=precip.sel(longitude=lon_west)
l_east['longitude']=[x-360. for x in lon_east]
precip=xr.concat([l_east,l_west],dim='longitude')

precip=precip.where(quality>=0.5)

defined1=xr.ones_like(precip).sum(dim='time')
defined=xr.ones_like(precip).where(precip>=0).sum(dim='time')
ratio_all=defined/defined1

valid=xr.ones_like(precip[0,:,:]).where(ratio_all>=0.8)

fig=plt.figure(figsize=(20,10))
ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())
ax.pcolormesh(precip.longitude,precip.latitude,valid,vmin=0,vmax=2,cmap=plt.cm.Greens)
ax.coastlines(resolution='10m')

g1=ax.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.5,color='gray',alpha=0.0,linestyle='--')

countries = cfeature.NaturalEarthFeature(category='cultural',name='admin_0_boundary_lines_land',scale='50m',facecolor='none')
ax.add_feature(countries)
g1.xlabel_style={'size':22,'color':'k'}
g1.ylabel_style={'size':22,'color':'k'}
g1.xformatter=LONGITUDE_FORMATTER
g1.yformatter=LATITUDE_FORMATTER

g1.top_labels=False
g1.right_labels=False
ax.set_xlim(-180,180)
ax.set_ylim(-60,70)

ax.set_title(r'$\bf{PRCPTOT}$'+' '+r'$\bf{90\%}$'+' '+r'$\bf{coverage}$'+' '+r'$\bf{1980-2018}$'+' ',fontsize=35,pad=5)
#ax.set_title(r'$\bf{R95p}$'+' '+r'$\bf{90\%}$'+' '+r'$\bf{coverage}$'+' '+r'$\bf{1901-2018}$'+' ',fontsize=35,pad=5)
            
plt.show()

In [ ]:
fig.savefig(dir_data+'hadex_3c.png')

In [ ]:
yrs=np.arange(1980,2019,1)

lats_sel=[x for x in ds.latitude.values if -60<=x<=70]
ratio_all=ratio_all.sel(latitude=lats_sel)

total_list_hadex3=[]

for i in range(len(yrs)):
    year=yrs[i]
    precip_sel=precip[i,:,:].sel(latitude=lats_sel).where(ratio_all>=0.8)
    
    if i==0:#land mask option
        ds1=xr.open_dataset('/Users/ahenny/Downloads/data.nc')
        t2m=ds1['t2m'][0,:,:]
        land_mask=t2m/t2m
        land_mask=land_mask.fillna(0)
        
        lon_east=[x for x in land_mask.longitude.values if x>=180.]
        lon_west=[x for x in land_mask.longitude.values if x<180.]
        l_east=land_mask.sel(longitude=lon_east)
        l_west=land_mask.sel(longitude=lon_west)
        l_east['longitude']=[x-360. for x in lon_east]
        land_mask=xr.concat([l_east,l_west],dim='longitude')
        land_mask_interp=land_mask.interp_like(precip_sel)

    weights=np.cos(np.deg2rad(precip_sel.latitude))
    total_weighted=precip_sel.where(land_mask_interp>=0.5).where(quality>=0.5).weighted(weights)
    #total_weighted=precip_sel.weighted(weights)
    mean=total_weighted.mean(dim=('latitude','longitude'),skipna=True).values.tolist()
    total_list_hadex3.append(mean)

In [ ]:
dk=xr.Dataset()
dk['starting_80']=(('time'),total_list_hadex3)
dk.coords['time']=np.arange(1980,2019,1)

try:
    os.remove(dir_data+'hadex3_lines_r99p_HQ.nc')
except OSError:
    pass
dk.to_netcdf(dir_data+'hadex3_lines_r99p_HQ.nc',mode='w',format='NETCDF4')

In [ ]:
test=total_list_hadex

yrs=np.arange(1901,2019,1)
r=st.linregress(yrs,test)
change=r[0]*len(yrs)
mean=float(sum(test))/float(len(test))
label_1='PRCPTOT 1901-2018 90% completeness (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'

test1=total_list_hadex1

r=st.linregress(np.arange(1950,2019,1),test1)
change=r[0]*len(yrs)
mean=float(sum(test1))/float(len(test1))
label_2='PRCPTOT 1950-2018 90% completeness (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'

#test2=total_list_hadex2

#r=st.linregress(np.arange(1925,2019,1),test2)
#change=r[0]*len(yrs)
#mean=float(sum(test2))/float(len(test2))
#label_3='PRCPTOT 1925-2018 90% completeness (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'


test3=total_list_hadex2

r=st.linregress(np.arange(1940,2019,1),test3)
change=r[0]*len(yrs)
mean=float(sum(test3))/float(len(test3))
label_4='PRCPTOT 1940-2018 90% completeness (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'


test4=total_list_hadex3

r=st.linregress(np.arange(1979,2019,1),test4)
change=r[0]*len(yrs)
mean=float(sum(test4))/float(len(test4))
label_5='PRCPTOT 1979-2018 90% completeness (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'



fig=plt.figure(figsize=(16,12))
ax=plt.subplot(1,1,1)
ax.plot(yrs,[x/1. for x in test],color='b',linewidth=6,label=label_1)
#ax.plot(np.arange(1925,2019,1),[x/1. for x in test2],color='cyan',linewidth=6,label=label_3)
ax.plot(np.arange(1950,2019,1),[x/1. for x in test1],color='limegreen',linewidth=6,label=label_2)
ax.plot(np.arange(1940,2019,1),[x/1. for x in test3],color='darkgreen',linewidth=6,label=label_4)

ax.plot(np.arange(1979,2019,1),[x/1. for x in test4],color='yellow',linewidth=6,label=label_5)

#ax.plot([yrs[0]+1./12.*x for x in range(len(yrs)*12)],total_list_hadex_monthly,color='r',linewidth=0.55)
ax.tick_params(labelsize=21)
ax.set_xlabel('Year',fontsize=23)
ax.set_ylabel('mm yr$^{-1}$',fontsize=23)
#ax.set_title(r'$\bf{Total}$'+' '+r'$\bf{precipitation}$',fontsize=34,pad=5)
ax.set_title(r'$\bf{PRCPTOT}$',fontsize=34,pad=5)
#ax.set_title(r'$\bf{R95p}$',fontsize=34,pad=5)
ax.grid(True)
plt.legend(loc='upper left',fontsize=18,ncol=1)
#ax.set_xlim(1978.8,2022.2)
ax.set_xlim(1900.3,2018.6)
#ax.set_ylim(115,250)
ax.set_ylim(510,910)
for axis in ['top', 'bottom', 'left', 'right']:
    ax.spines[axis].set_linewidth(1.75)
#ax.fill_between(np.arange(1978,1991,1),200,1150,color='gray',alpha=0.3)
#ax.fill_between(np.arange(2021,2024,1),200,1150,color='gray',alpha=0.3)
plt.show()


In [ ]:
fig.savefig(dir_data+'hadex_2.png')